<a href="https://colab.research.google.com/github/gomesluiz/pln-na-pratica/blob/main/03_a_representa%C3%A7%C3%A3o_textual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# instalação dos pacotes necessários
!pip install nltk==3.8.1
!pip install gensim==4.3.2
!pip install umap-learn==0.5.5
!pip install wikipedia==1.4.0
!pip install unidecode==1.3.8

In [18]:
# Importações da biblioteca padrão
import bz2
import datetime
import os
import re
import string
import sys
import urllib.request
import warnings
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# Importações de bibliotecas de terceiros
import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
#import umap
import wikipedia
from gensim.models import word2vec
from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize
#from nltk.util import ngrams
#from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.manifold import TSNE
from unidecode import unidecode

# Downloads do NLTK
nltk.download('punkt')
nltk.download("stopwords")

# Configurações e comandos específicos (por exemplo, desativar avisos)
warnings.filterwarnings('ignore')
%matplotlib inline

print("Pacotes importados com sucesso; notebook pronto para uso!")

Pacotes importados com sucesso; notebook pronto para uso!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# declara as funções utilitárias do notebook
def formata_msg(nivel, msg):
  """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp
    e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.

    Retorna:
    - str: A mensagem de log formatada.
  """

  timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

  return f"[{nivel}] {timestamp} - {msg}"


def baixar_extrair(modelo, arquitetura):
    """
    Baixa e extrai um arquivo zip de embeddings de um modelo e arquitetura
    específicos de um servidor remoto.

    Args:
        modelo (str): O nome do modelo de embeddings.
        arquitetura (str): A arquitetura específica do modelo.

    Returns:
        str: O caminho para o arquivo extraído.
    """
    url = f"http://143.107.183.175:22980/download.php?file=embeddings/{modelo}/{arquitetura}.zip"
    caminho_pasta_saida = os.path.join("embeddings", modelo)
    caminho_arquivo_saida = os.path.join(caminho_pasta_saida, arquitetura)
    print(formata_msg("INFO", f"Baixando: {modelo}_{arquitetura}"))
    with urlopen(url) as resposta:
        with ZipFile(BytesIO(resposta.read())) as arquivo_zip:
            arquivo_zip.extractall(caminho_pasta_saida)

    return os.path.join(caminho_pasta_saida, os.listdir(caminho_pasta_saida)[0])

print(formata_msg("INFO", "Funções utilitárias prontas para utilização."))
print(formata_msg("INFO", f"Versão do Python: {sys.version} "))

[INFO] 2024-03-27 03:05:10 - Funções utilitárias prontas para utilização.
[INFO] 2024-03-27 03:05:10 - Versão do Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] 


# Definição do Corpus

Selecionamos algumas frases do corpus de BH da wikipedia.

Conside a lista abaixo como nosso corpus de documentos. Cada elemento da lista, considere como um único documento.

In [24]:
documentos = \
["Belo Horizonte é um município brasileiro e a capital do estado de Minas Gerais",
"A populacao de Belo Horizonte é estimada em 2 501 576 habitantes, conforme estimativas do Instituto Brasileiro de Geografia e Estatística",
"Belo Horizonte já foi indicada pelo Population Crisis Commitee, da ONU, como a metrópole com melhor qualidade de vida na América Latina",
"Belo Horizonte é mundialmente conhecida e exerce significativa influência nacional e até internacional, seja do ponto de vista cultural, econômico ou político",
"Belo Horizonte é a capital do segundo estado mais populoso do Brasil, Minas Gerais"]

## Preprocessamento

<b> Atividade </b>

1) Escreva uma método que realiza o pré-processamento da lista de <b>documentos</b>.

O método deve, para cada documento:
- tokenizar cada palavra
- remover stopwords
- remover números
- remover pontuções
- remover acentos

In [23]:
def pre_processa_texto(texto):
    """
    Preprocessa o texto fornecido realizando várias etapas de limpeza.

    Etapas:
    1. Tokeniza o texto.
    2. Converte os tokens para minúsculos.
    3. Remove stopwords em português.
    4. Remove números dos tokens.
    5. Exclui tokens que são pontuações.
    6. Remove acentuações dos tokens.

    Parâmetros:
    texto (str): O texto a ser preprocessado.

    Retorna:
    list: Lista de tokens preprocessados.
    """

    # Tokeniza o texto usando um padrão para capturar palavras e pontuações.
    padrao = r"\w+(?:'\w+)?|[^\w\s]"
    tokens_preprocessados = re.findall(padrao, texto)

    # Converte os tokens para minúsculos para padronizar a capitalização.
    tokens_preprocessados = [token.lower() for token in tokens_preprocessados]

    # Remove stopwords para reduzir o conjunto de tokens a palavras significativas.
    portugues_stops = stopwords.words('portuguese')
    tokens_preprocessados = [token for token in tokens_preprocessados
                             if token not in portugues_stops]

    # Remove números, pois geralmente não contribuem para o significado do texto.
    tokens_preprocessados = [re.sub(r'\d+', '', token) for token in tokens_preprocessados
                             if re.sub(r'\d+', '', token)]

    # Exclui tokens que são pontuações, pois raramente são úteis para análise de texto.
    tokens_preprocessados = [token for token in tokens_preprocessados
                             if token not in string.punctuation]

    # Remove acentuações para padronizar os tokens.
    tokens_preprocessados = [unidecode(token) for token in tokens_preprocessados]

    return ' '.join(tokens_preprocessados)


In [25]:
documentos_preprocessados = [pre_processa_texto(documento) for documento in documentos]

# Representação Textual

### Phrases - Gensim

Forma mais inteligente de calcular os bigrams. Ela calcula os bigramas levando em consideração a frequência do par das palavaras em todos os documentos.
Para isso ele treina um modelo e depois aplica no corpus.

```python
#treinamento bigrams
model_corpus_phrases = gensim.models.Phrases(corpus_processado, min_count=1)
#calulando os bigrams do corpus processado
bigram_corpus = model_corpus_phrases[corpus_processado]
```

<b> Atividade </b>

2) Faça um código que treine os bigrams, sendo que o <b>min_count = 1</b>.
O <b>min_count</b> é a contagem mínima que aquele par de palavras deve aparecer junto para considerarmos com um token. Teste também com outros valores de mim_count. Depois imprima os bigramas de cada documento.
Use o corpus_processado.

In [28]:
# treinamento bigrams
model_corpus_phrases = gensim.models.Phrases(documentos_preprocessados, min_count=1)
# calulando os bigrams do corpus processado
bigram_corpus = model_corpus_phrases[documentos_preprocessados]
print(bigram_corpus)

['belo horizonte municipio brasileiro capital estado minas gerais', 'populacao belo horizonte estimada habitantes conforme estimativas instituto brasileiro geografia estatistica', 'belo horizonte indicada population crisis commitee onu metropole melhor qualidade vida america latina', 'belo horizonte mundialmente conhecida exerce significativa influencia nacional internacional ponto vista cultural economico politico', 'belo horizonte capital segundo estado populoso brasil minas gerais']


## TD-IDF

1) Faça o TDIFTVectorizer nos documentos da variável <b>documentos</b> sem alterar nenhum parâmetro.

In [29]:
# Inicializa o TfidfVectorizer com parâmetros específicos para
# limitar a frequência máxima e mínima dos termos nos documentos.
vectorizer = TfidfVectorizer(min_df=2, max_df=0.5)

# Transforma os documentos em uma matriz TF-IDF.
tfidf_matrix = vectorizer.fit_transform(documentos_preprocessados)

# Imprime a forma da matriz TF-IDF para mostrar o número de documentos
# e o tamanho do vocabulário.
print(formata_msg("INFO", f"Forma da matriz TF-IDF: {tfidf_matrix.shape}"))

# Obtém o vocabulário extraído dos documentos e imprime.
vocabulario = vectorizer.get_feature_names_out()
print(formata_msg("INFO", f"Vocabulário:{vocabulario}"))

# Cria um DataFrame para os valores TF-IDF de cada termo por documento.
df = pd.DataFrame(
        tfidf_matrix.T.todense(),
        index=vectorizer.get_feature_names_out(),
        columns=["doc" + str(i + 1) for i in range(len(documentos))]
    )
print(formata_msg("INFO", f"Dataframe:\n\n{df}"))

[INFO] 2024-03-27 03:15:19 - Forma da matriz TF-IDF: (5, 5)
[INFO] 2024-03-27 03:15:19 - Vocabulário:['brasileiro' 'capital' 'estado' 'gerais' 'minas']
[INFO] 2024-03-27 03:15:19 - Dataframe:

                doc1  doc2  doc3  doc4  doc5
brasileiro  0.447214   1.0   0.0   0.0   0.0
capital     0.447214   0.0   0.0   0.0   0.5
estado      0.447214   0.0   0.0   0.0   0.5
gerais      0.447214   0.0   0.0   0.0   0.5
minas       0.447214   0.0   0.0   0.0   0.5


## Bag of Words

Faça o CountVectorizer nos documentos da variável <b>documentos</b> considerando binary = True

In [ ]:
# Inicializa o TfidfVectorizer com parâmetros específicos para
# limitar a frequência máxima e mínima dos termos nos documentos.
vectorizer = CountVectorizer(binary=True)

# Transforma os documentos em uma matriz TF-IDF.
bow_matrix = vectorizer.fit_transform(documentos_preprocessados)

# Imprime a forma da matriz TF-IDF para mostrar o número de documentos
# e o tamanho do vocabulário.
print(formata_msg("INFO", f"Forma da matriz TF-IDF: {bow_matrix.shape}"))

# Obtém o vocabulário extraído dos documentos e imprime.
vocabulario = vectorizer.get_feature_names_out()
print(formata_msg("INFO", f"Vocabulário:{vocabulario}"))

# Cria um DataFrame para os valores TF-IDF de cada termo por documento.
df = pd.DataFrame(
        bow_matrix.T.todense(),
        index=vectorizer.get_feature_names_out(),
        columns=["doc" + str(i + 1) for i in range(len(documentos))]
    )
print(formata_msg("INFO", f"Dataframe:\n\n{df}"))

## Embedding

### Utilizando um embedding treinado

9) Faça download do seguinte arquivo, realize a leitura deste arquivo e carregue o modelo do Repositório de Word Embeddings do NILC:
http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc#



In [30]:
%%time
modelo, arquitetura="word2vec", "cbow_s50"
caminho_arquivo_completo = baixar_extrair(modelo, arquitetura )
print(caminho_arquivo_completo)

[INFO] 2024-03-27 03:16:59 - Baixando: word2vec_cbow_s50
embeddings/word2vec/cbow_s50.txt
CPU times: user 4.02 s, sys: 2.42 s, total: 6.44 s
Wall time: 23.4 s


In [31]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(caminho_arquivo_completo, binary=False)

<b> Atividade </b>

10) Imprima os vetores das palavras "nlp" e "computacao"

```python
#exemplo de retorno do vetor
word_vectors[__]
```

In [ ]:
word_vectors.key_to_index

In [ ]:
word_vectors['nlp']

<b> Similaridade de Vetores </b>

No gensim é possível realizar a similaridade utilizando o seguinte método:

```python
word_vectors.most_similar(___)
```

<b> Atividade </b>

11) Verifique a similaridade das seguintes palavras: elizabete, raiva, segunda, dois, computação.

In [ ]:
word_vectors.most_similar("elizabete")

In [ ]:
word_vectors.most_similar("raiva")

In [ ]:
word_vectors.most_similar("segunda")

In [ ]:
word_vectors.most_similar("computação")